# 📊 Grafisk Risikomatrise - NeqSim

Denne notebooken viser hvordan man kan:
1. **Definere risiko for utstyr** (sannsynlighet for feil)
2. **Simulere konsekvenser** (produksjonstap når utstyr går ned)
3. **Beregne kostnader** for ulike risikoscenarier
4. **Visualisere i risikomatrise** (5x5 matrise med fargekoding)

In [ ]:
# Import NeqSim - Direct Java Access via jneqsim
from neqsim import jneqsim
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

# Import Java classes through the jneqsim gateway
SystemSrkEos = jneqsim.thermo.system.SystemSrkEos
ProcessSystem = jneqsim.process.processmodel.ProcessSystem
Stream = jneqsim.process.equipment.stream.Stream
Separator = jneqsim.process.equipment.separator.Separator
Compressor = jneqsim.process.equipment.compressor.Compressor
Cooler = jneqsim.process.equipment.heatexchanger.Cooler
Pump = jneqsim.process.equipment.pump.Pump
ThrottlingValve = jneqsim.process.equipment.valve.ThrottlingValve
RiskMatrix = jneqsim.process.safety.risk.RiskMatrix

print("✅ NeqSim Risikomatrise lastet!")

## 1. Bygg Prosessanlegg

Vi lager et typisk gassbehandlingsanlegg med flere utstyrsenheter.

In [ ]:
# Lag fluid
fluid = SystemSrkEos(280.0, 50.0)
fluid.addComponent("methane", 0.82)
fluid.addComponent("ethane", 0.10)
fluid.addComponent("propane", 0.05)
fluid.addComponent("n-butane", 0.03)
fluid.setMixingRule("classic")

# Bygg prosess
process = ProcessSystem()

feed = Stream("Brønnstrøm", fluid)
feed.setFlowRate(15000.0, "kg/hr")  # 15 tonn/time
feed.setTemperature(40.0, "C")
feed.setPressure(40.0, "bara")
process.add(feed)

# Innløpsseparator
separator = Separator("HP Separator", feed)
process.add(separator)

# Kompressor
compressor = Compressor("Eksportkompressor", separator.getGasOutStream())
compressor.setOutletPressure(120.0, "bara")
process.add(compressor)

# Kjøler
cooler = Cooler("Eksportkjøler", compressor.getOutletStream())
cooler.setOutTemperature(30.0, "C")
process.add(cooler)

# Kondensatpumpe
pump = Pump("Kondensatpumpe", separator.getLiquidOutStream())
pump.setOutletPressure(50.0, "bara")
process.add(pump)

# Eksportstrøm
export = Stream("Eksportgass", cooler.getOutletStream())
process.add(export)

process.run()
print(f"✅ Prosess kjørt - Eksportproduksjon: {export.getFlowRate('kg/hr'):.0f} kg/hr")

## 2. Definer Risikodata for Utstyr

Her setter vi inn **sannsynlighet** (feil/år) og **reparasjonstid** (MTTR) for hvert utstyr.

In [ ]:
# Lag risikomatrise
matrix = RiskMatrix(process)
matrix.setFeedStreamName("Brønnstrøm")
matrix.setProductStreamName("Eksportgass")

# Sett økonomiske parametere
matrix.setProductPrice(4000.0, "NOK/tonne")  # Gasspris
matrix.setDowntimeCostPerHour(50000.0)  # Faste kostnader per time nedetid

# Definer risiko for hvert utstyr (feil/år, MTTR i timer)
# Høyere tall = høyere risiko
matrix.addEquipmentRisk("Eksportkompressor", 0.8, 72)   # Kompressor: 0.8 feil/år, 72t reparasjon
matrix.addEquipmentRisk("Eksportkjøler", 0.3, 24)       # Kjøler: 0.3 feil/år, 24t reparasjon
matrix.addEquipmentRisk("HP Separator", 0.1, 48)        # Separator: 0.1 feil/år, 48t reparasjon
matrix.addEquipmentRisk("Kondensatpumpe", 1.5, 36)      # Pumpe: 1.5 feil/år, 36t reparasjon

print("✅ Risikodata definert")
print("\n📋 Utstyrsrisiko:")
print("  Eksportkompressor: 0.8 feil/år, 72t MTTR")
print("  Eksportkjøler: 0.3 feil/år, 24t MTTR")
print("  HP Separator: 0.1 feil/år, 48t MTTR")
print("  Kondensatpumpe: 1.5 feil/år, 36t MTTR (høy risiko!)")

## 3. Simuler Konsekvenser med NeqSim

NeqSim simulerer nå hva som skjer med produksjonen når hvert utstyr går ned.

In [ ]:
# Bygg risikomatrise - NeqSim simulerer konsekvenser
matrix.buildRiskMatrix()

# Vis resultat som tabell
risk_data = []
for assessment in matrix.getRiskAssessmentsSortedByRisk():
    risk_data.append({
        'Utstyr': assessment.getEquipmentName(),
        'Feil/År': f"{assessment.getFailuresPerYear():.2f}",
        'MTTR (t)': f"{assessment.getMttr():.0f}",
        'Prodtap %': f"{assessment.getProductionLossPercent():.1f}",
        'P-Kat': assessment.getProbabilityCategory().getName(),
        'K-Kat': assessment.getConsequenceCategory().getName(),
        'Risiko': assessment.getRiskLevel().getName(),
        'Score': assessment.getRiskScore(),
        'Årskost (NOK)': f"{assessment.getAnnualRiskCost():,.0f}"
    })

df = pd.DataFrame(risk_data)
print("\n📊 RISIKOOVERSIKT")
print("=" * 100)
print(df.to_string(index=False))
print("\n" + "=" * 100)
print(f"💰 TOTAL ÅRLIG RISIKOKOSTNAD: {matrix.getTotalAnnualRiskCost():,.0f} NOK")

## 4. Grafisk Risikomatrise (5x5)

Visualiserer risiko med fargekoding:
- 🟢 **Grønn** = Lav risiko
- 🟡 **Gul** = Medium risiko  
- 🟠 **Oransje** = Høy risiko
- 🔴 **Rød** = Kritisk risiko

In [ ]:
def plot_risk_matrix(matrix):
    """Plott 5x5 risikomatrise med utstyr."""
    
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Definer fargematrise (5x5) - Sannsynlighet(x) vs Konsekvens(y)
    # Lav=grønn, Medium=gul, Høy=oransje, Kritisk=rød
    colors = np.array([
        ['#90EE90', '#90EE90', '#FFFF00', '#FFA500', '#FF4500'],  # Konsekvens 1 (Negligible)
        ['#90EE90', '#FFFF00', '#FFFF00', '#FFA500', '#FF4500'],  # Konsekvens 2 (Minor)
        ['#FFFF00', '#FFFF00', '#FFA500', '#FFA500', '#FF0000'],  # Konsekvens 3 (Moderate)
        ['#FFA500', '#FFA500', '#FFA500', '#FF0000', '#FF0000'],  # Konsekvens 4 (Major)
        ['#FFA500', '#FF0000', '#FF0000', '#FF0000', '#FF0000'],  # Konsekvens 5 (Catastrophic)
    ])
    
    # Tegn bakgrunnsfarger
    for i in range(5):
        for j in range(5):
            rect = mpatches.Rectangle((j, i), 1, 1, 
                                       linewidth=2, 
                                       edgecolor='white', 
                                       facecolor=colors[i, j],
                                       alpha=0.7)
            ax.add_patch(rect)
    
    # Hent matrisepunkter
    matrix_data = list(matrix.getMatrixData())
    
    # Plott utstyr som sirkler
    for point in matrix_data:
        x = float(point.get('x')) - 0.5  # Sentrer i celle
        y = float(point.get('y')) - 0.5
        cost = float(point.get('annualCost'))
        name = str(point.get('name'))
        
        # Størrelse basert på kostnad
        size = 200 + cost / 5000
        
        # Plott punkt
        ax.scatter(x, y, s=size, c='navy', edgecolors='white', 
                  linewidths=2, zorder=10, alpha=0.9)
        
        # Legg til navn
        ax.annotate(name, (x, y), 
                   xytext=(5, 5), textcoords='offset points',
                   fontsize=9, fontweight='bold',
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    
    # Akser
    ax.set_xlim(0, 5)
    ax.set_ylim(0, 5)
    
    # Labels
    prob_labels = ['Svært\nlav', 'Lav', 'Medium', 'Høy', 'Svært\nhøy']
    cons_labels = ['Neglisjerbar', 'Mindre', 'Moderat', 'Stor', 'Katastrofal']
    
    ax.set_xticks([0.5, 1.5, 2.5, 3.5, 4.5])
    ax.set_xticklabels(prob_labels, fontsize=10)
    ax.set_yticks([0.5, 1.5, 2.5, 3.5, 4.5])
    ax.set_yticklabels(cons_labels, fontsize=10)
    
    ax.set_xlabel('SANNSYNLIGHET (feil/år)', fontsize=12, fontweight='bold')
    ax.set_ylabel('KONSEKVENS (produksjonstap)', fontsize=12, fontweight='bold')
    ax.set_title('📊 RISIKOMATRISE - Utstyrssvikt\n(størrelse indikerer årlig kostnad)', 
                fontsize=14, fontweight='bold')
    
    # Legende
    legend_elements = [
        mpatches.Patch(facecolor='#90EE90', label='Lav risiko'),
        mpatches.Patch(facecolor='#FFFF00', label='Medium risiko'),
        mpatches.Patch(facecolor='#FFA500', label='Høy risiko'),
        mpatches.Patch(facecolor='#FF0000', label='Kritisk risiko'),
    ]
    ax.legend(handles=legend_elements, loc='upper left', fontsize=10)
    
    plt.tight_layout()
    plt.grid(True, alpha=0.3)
    return fig

# Plott matrisen
fig = plot_risk_matrix(matrix)
plt.show()

## 5. Kostnadsoversikt (Pareto)

Viser hvilke utstyr som bidrar mest til risikokostnaden.

In [ ]:
# Lag kostnadsdiagram
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Data
assessments = list(matrix.getRiskAssessmentsSortedByCost())
names = [a.getEquipmentName() for a in assessments]
costs = [a.getAnnualRiskCost() for a in assessments]
colors = [a.getRiskLevel().getColor() for a in assessments]

# Pareto (kostnad)
bars = ax1.barh(names, costs, color=colors, edgecolor='black')
ax1.set_xlabel('Årlig risikokostnad (NOK)', fontsize=11)
ax1.set_title('💰 Risikokostnad per Utstyr', fontsize=12, fontweight='bold')
ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))

# Legg til verdier
for bar, cost in zip(bars, costs):
    ax1.text(bar.get_width() + max(costs)*0.02, bar.get_y() + bar.get_height()/2,
             f'{cost:,.0f}', va='center', fontsize=9)

# Risikoscore
scores = [a.getRiskScore() for a in assessments]
bars2 = ax2.barh(names, scores, color=colors, edgecolor='black')
ax2.set_xlabel('Risikoscore (P x K)', fontsize=11)
ax2.set_title('⚠️ Risikoscore per Utstyr', fontsize=12, fontweight='bold')

# Risikogrenser
ax2.axvline(x=4, color='green', linestyle='--', alpha=0.7, label='Lav grense')
ax2.axvline(x=9, color='orange', linestyle='--', alpha=0.7, label='Medium grense')
ax2.axvline(x=15, color='red', linestyle='--', alpha=0.7, label='Høy grense')
ax2.legend(loc='lower right', fontsize=8)

plt.tight_layout()
plt.show()

# Oppsummering
print(f"\n📈 OPPSUMMERING")
print(f"="*50)
print(f"Total årlig risikokostnad: {matrix.getTotalAnnualRiskCost():,.0f} NOK")
print(f"Kritisk risiko: {len(matrix.getEquipmentByRiskLevel(RiskMatrix.RiskLevel.CRITICAL))} utstyr")
print(f"Høy risiko: {len(matrix.getEquipmentByRiskLevel(RiskMatrix.RiskLevel.HIGH))} utstyr")
print(f"Medium risiko: {len(matrix.getEquipmentByRiskLevel(RiskMatrix.RiskLevel.MEDIUM))} utstyr")
print(f"Lav risiko: {len(matrix.getEquipmentByRiskLevel(RiskMatrix.RiskLevel.LOW))} utstyr")

## 6. Detaljert JSON-Rapport

Full rapport som kan brukes til IOC-annotering eller videre analyse.

In [ ]:
# Full JSON rapport
report = json.loads(matrix.toJson())
print(json.dumps(report, indent=2, ensure_ascii=False))

## 📝 Konklusjon

Denne analysen viser:

1. **Sannsynlighet** - Fra OREDA-data eller egne inndata
2. **Konsekvens** - NeqSim simulerer produksjonstapet
3. **Risikokostnad** - Beregner årlig forventet tap
4. **Prioritering** - Viser hvilke utstyr som krever oppmerksomhet

### Neste steg:
- Vurder tiltak for utstyr med **høy/kritisk** risiko
- Evaluer ROI på redundans/spare-parts
- Integrer med vedlikeholdsplanlegging